## Lecture 1

[official lecture notes](https://timroughgarden.github.io/fob21/l/l1.pdf) 

[Archive](https://web.archive.org/web/20230000000000*/https://timroughgarden.github.io/fob21/l/l1.pdf)

### Blockchain stack
Not a serious taxonomy. It is also fuzzy. In reality, the separation between the layers is not as clean (status in late 2021).

Bottom -> Top

0. Layer 0: The Internet (untrusted parties can send arbitrary packets), it is not perfect (delays, malwares etc.)
1. Layer 1: Consensus Layer (IMP Layer) Keeping the scattered nodes in sync. (e.g., Etherium, Bitcoin)
2. Layer 1.5: Compute Layer (smart contracts) (agreement on the sequence of instruction, but also execution of these instructions).
3. Layer 2: Scaling Layer. Same functionality of layer 1, but at scale. (Bitcoins 5 transactions per second, this is slow).
4. Layer 3: Application Layer. 

### Overview

- 60% of the lectures are about Layer 1.
    - classical protocols
    - Naka moto/longest chain consensus
    - sybil-resistance algorithm
    - difficulty adjustment algorithms
    - deep dives in Bitcoin, Ethereum (not engineering details, just theory)
- 20% of the lectuers about Layer 2.
    - lightning network
    - rollouts (Etherium)
    - next gen layer 1 protocols
- Applications layer
    - DeFi (Decentralized Finance)
    - if time permits, MEV (Miner Extractable Value) and DAOS

### Digital Signature Schemes

Consensus: Keep multiple nodes in sync.

Q: Why difficult?
A: DDoS, delays, out of date protocol, malicious protocol

Lecture 2 will start with too many restrictions. Upcoming lectuers will relax most of them, but two.
1. Internext exists
2. Cryptography exists

DSS is made of three algorithms.
1. Key generation algorithm (maps random seed to public key, secret key pair) e.g., ssh-keygen
2. signing algorithm (maps msg + secret key to a msg + signature)
3. Verification algorithm (maps msg+signature+public key to boolean)

We will assume that signatures are ideal, i.e., forgery is impossible). Not 100% accurate, as in principle, we can bruteforce. Bruteforce is exponential. How do we know that a clever polynomial algorithm does not exist? We assume that there is not. 

Unless otherwise noted, assume that every message in the protocols have signatures attached to it.

### SMR(State Machine Replication) Problem 

Problem worked in 1980s. State machines are similar to finite automata. 

State machine:

After executing instruction, state changes. (In 1980s state was state of the database, insert delete changes the database).(Nowadays, it can be current balance, all current users etc.)

Replication:

In 1980s, people wanted to provide higher uptime for databases. The solution was to have multiple copies of the database, in different machines. But, the problem is how to keep the state between them synced?

In Blockchain, we have decentralized system. All the nodes are replicating instruction execution. No one is the sole owner of the state. 

Nodes: Running the protocols
Clients: Users (read the state or change the state)

Messages from clients to nodes are referred as transactions.

each node maintains a local append-only data structure (aka history). History is ordered. Transactions are stored into this history. We want to keep the history syncronized.

Goals for the solution
- consistency: keep the histories synced (same txs sequence), some machines are allowed to lag behind. (any pair of nodes, either both of them have identical histories, or one of their history is a prefix of another one).
- liveness: every valid tx submitted eventually added to history.

Next 4 lectures solve this problem for various assumptions. 


## Overview of Lectuers 2-7

- [RG Notes](https://timroughgarden.github.io/fob21/l/l2-7-overview.pdf)  
- [Archive](https://web.archive.org/web/20220131143244/https://timroughgarden.github.io/fob21/l/l2-7-overview.pdf)
- [RG Slides](https://timroughgarden.github.io/fob21/slides/F21L2-7-overview.pdf)

- consensus (discussed in lec 1)
- consensus from first principles (l2-l7) classic material on consensus
- remaining lectures are about last 5 years (till 2021)
- possibility results and impossibility results (good protocols do not exists unless certain assumptions are made)

- L7. Real protocol (Tendermint protocol (used by cosmos and terra)).
  - Provable guarantees.
    - consistency
    - liveness
- L6. Partial synchronus model. (IMP for blockchain)
    - normal mode
    - attack mode
    - Will prove 67% honest nodes are required.
    - Relate above things with CAP theorem.
    - Partial model is interpolation of ...
- L2, L3: Synchronous protocol
- L4, L5: Asynchronous protocol



## Lecture 2: The Dolev-Strong Protocol
- [RG Notes](https://timroughgarden.github.io/fob21/l/l2.pdf)
- [RG Notes Archive](https://web.archive.org/web/20220204231653/https://timroughgarden.github.io/fob21/l/l2.pdf)

SMR Problem: State Machine Replication

- Nodes: machines that are running the protocol 
- Clients: submit the transactions (users)

History: Append only **ordered** sequence of the transactions.

Goal: A protocol (event-driven code) satisfies 
1. Consistency: All nodes agree on the same history 
2. Liveness: Every submitted transactions eventually get added to all the node histories

Solution depends on the assumptions we make.

Assumptions (to be relaxed)
1. Permissioned setting: A priori known set of nodes {1, 2, 3, ..., n}. (can be identified by IP addresses)
2. PKI (public key infra.) Each node i has pk_i and sk_i pairs. All nodes know everybody elses' public keys.
3. Synchronous model: 
- All of the nodes share a global clock.  
- (Very strong assumption) All messages sent at time t, arrive by time t+1.
4. All the nodes are honest (all the nodes run intended protocols). Will be the first one to be relaxed.

Under these assumption, we can solve an SMR which has consistency and liveness.


### Solution via Round Robin Leaders

- Nodes take turns as leaders. (i.e., at time t node t%n is the leader). This leader does the coordination. 
- The leader orders the message it receives and sends them to all other nodes. (Plays the role of a block in blockchain).
  - Because, we are in synchronous setting, everyone receives the message.
- When node receives the recent leader's ordered list, it appends it to it's local history.
- This protocol satisfies consistency and liveness.
  - Consistency is obvious. (Everyone receives same message from the reader, and everyone is honest).
  - Liveness. (every node receives their chance to send their transactions will be added by everyone else, rotating leader is require here. Uniformly random leader could also have worked.)

### Faulty Nodes (not all nodes are honest)

A node can fail to send their messages, because they were down. These kind of noes are called as faulty.

#### Types of the faults
1. Crash fault: machine goes down. Runs honestly till failure.
2. Omission fault: Faulty node may miss some of the messages it was supposed to send. (It is atleast as severe as crash fault. Omission can miss **some** of the messages.) It creates a trouble for consistency. It may send correct message to some nodes, but miss to send out to other nodes. Thus, committed messages diverge.
3. Byzantine Fault: no assumptions about their behavior, except they can't break cryptography. The nodes can lie differently to different nodes.

#### Relaxation
- for a known parameter f, <= f nodes are Byzantine.
- But, we do not know who are the liers. If we did, we would just ignore them.
- We also assume that the Byzantine nodes are static.


#### Byzantine Broadcast problem

Idea is still the same of rotating leaders approach.

sender has a private input $v^* \in V$. No one else knows about this message.

Goals (only applicable for honest nodes):
- Termination: All honest node eventually halt with some valuen $v_i \in V$. 
- Agreement: All honest nodes choose the same value.
- Validity: If the sender is honest, all honest $v_i = v^*$.

subroutine: Byzantine Broadcas (BB)


#### BB can solve SMR

1. take turns as leader
2. run BB protocol (with sender = current leader), agree on a tx list L. 
3. Each node committs L to their history.

Question to ponder: Why does it solve consistency and liveness?

Hint:
- BB agreement => SMR consistency.
- BB validity => SMR liveness.

#### Intuition for Dolev-Strong (Case: f=1, n > 3)


- at t=0, sender sends signed message $v^*$ to every one.
- at t=1, everyone(but the sender) echoes signed message they received, (this echo message is also signed).
- at t=2, use majority voting to figureout the correct message (break tie in consistent manner)
  - <= 1 vote from the sender (sender may be Byzantine, so inequality)
  - <= n-2 vote from non-senders (message is not sent to self, and some(f=1) Byzantine node didn't send the message.)
 
Try to see why this protocol satisfies termination, validity and agreement. Also try to see what happens when there are two Byzantine nodes. (Hint: They can collude). See linked notes for the answers.

### The Dolev Strong Protocol

Note: this protocol relies on synchronous model and is slow.